In [ ]:
'''匯入套件'''
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import json, os, pprint, time
from urllib import parse

'''設定 Chrome 瀏覽器開啟時的狀態'''
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--incognito')
options.add_argument('--disable-popup-blocking')

'''建立操控 Chrome 瀏覽器的變數'''
# 使用 Chrome 的 WebDriver
driver = webdriver.Chrome(
    options = my_options,
    service = Service(ChromeDriverManager().install())
)

'''放置 金庸小說 metadata 的資訊'''
listData = []

'''小庸小說的網址'''
url = 'https://www.bookwormzz.com/zh/'

# 沒有放置 txt 檔的資料夾，就建立起來
folderPath = 'jinyong'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

# 走訪預設頁面
def visit():
    driver.get(url)

# 取得小說的主要連結
def getMainLinks():
    a_elms = driver.find_elements(By.CSS_SELECTOR, 'a[data-ajax="false"]')
    for a in a_elms:
        listData.append({
            "title": a.get_attribute('innerText'),
            "link": parse.unquote(a.get_attribute('href')) + "#book_toc"
        })

# 取得所有小說的獨立連結
def getSubLinks():
    for i in range( len(listData) ):
        # 沒有 sub 屬性，則建立，為了放置各個章回小說的 metadata
        if "sub" not in listData[i]:
            listData[i]["sub"] = []
        
        driver.get(listData[i]["link"])

        # 若是走訪網頁時，選擇不到特定的元素，視為沒有資料，continue 到 for 的下一個 index 去
        try:
            WebDriverWait(driver, 3).until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, 'a[rel="external"][class="ui-link"]')
                )
            )

            a_elms = driver.find_elements(By.CSS_SELECTOR, 'a[rel="external"][class="ui-link"]')
            for a in a_elms:
                listData[i]["sub"].append({
                    "sub_title": a.get_attribute("innerText"),
                    "sub_link": parse.unquote( a.get_attribute("href") )
                })
        except TimeoutException as e:
            continue

# 建立 金庸小說 metadata 的 json 檔
def saveJson():
    fp = open(f"{folderPath}/jinyong.json", "w", encoding="utf-8")
    fp.write( json.dumps(listData, ensure_ascii=False) )
    fp.close()

# 將金庸小說所有章回的內容，各自寫到 txt 與 json 中
def writeTxt():
    # 稍候建立 train.json 前的程式變數
    listContent = []

    # 開啟 金庸小說 metadata 的 json 檔
    fp = open(f"{folderPath}/jinyong.json", "r", encoding="utf-8")
    strJson = fp.read()
    fp.close()

    # 走訪所有章回的小說文字內容
    listResult = json.loads(strJson)
    for i in range( len(listResult) ):
        for j in range( len(listResult[i]["sub"]) ):
            driver.get( listResult[i]['sub'][j]['sub_link'] )
            div = driver.find_element(By.CSS_SELECTOR, 'div#html > div')
            strContent = div.get_attribute('innerText')
            
            # 資料清理
            strContent = re.sub(r" |\r|\n|　|\s", '', strContent)

            # 決定 txt 的檔案名稱
            fileName = f"{listResult[i]['title']}_{listResult[i]['sub'][j]['sub_title']}.txt"

            # 將小說內容存到 txt 中
            fp = open(f"{folderPath}/{fileName}", "w", encoding="utf-8")
            fp.write( strContent )
            fp.close()

            # 額外將小說內容放到 list 當中，建立 train.json
            listContent.append(strContent)

    # 延伸之後的教學，在此建立訓練資料
    fp = open(f"{folderPath}/train.json", "w", encoding="utf-8")
    fp.write( json.dumps(listContent, ensure_ascii=False) )
    fp.close()

# 關閉瀏覽器
def close():
    driver.quit()

if __name__ == "__main__":
    time1 = time.time()
    visit()
    getMainLinks()
    getSubLinks()
    saveJson()
    writeTxt()
    close()
    print(f"執行總花費時間: {time.time() - time1}")